In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -y psi4 python=3.7 -c psi4

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

# this command is needed to avoid "Loader" error.
!pip install distributed

--2022-04-20 13:16:04--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-20 13:16:04--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M  81.2MB/s    in 0.9s    

2022-04-20 13:16:05 (81.2 MB/s) - ‘Miniconda3-latest-Linu

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [14]:
psi4.set_output_file("log.txt")

psi4.set_memory('10GB')
psi4.set_num_threads(2)

mol = psi4.geometry("""
symmetry c1
0 1
O       -1.0239104333      1.3152089264     -0.0149071027                 
H       -0.0345840622      1.2807129464      0.0105245830                 
H       -1.3132760473      0.5529382075      0.5470664770                 
""")

#scf_e, wfn = psi4.optimize("B3LYP-D3BJ/6-311G(d,p)",  return_wfn = True)
#scf_e, wfn = psi4.optimize("M06-D3/6-311G(d,p)", return_wfn = True) # metal
scf_e, wfn = psi4.optimize("M06-2X-D3/6-311G(d,p)", return_wfn = True) # non-metal

Optimizer: Optimization complete!


In [15]:
!cat log.txt


  Memory set to   9.313 GiB by Python driver.
  Threads set to 2 by Python driver.

Scratch directory: /tmp/

Scratch directory: /tmp/
gradient() will perform analytic gradient computation.

*** tstart() called on 0140d0042e2e
*** at Wed Apr 20 13:24:55 2022

   => Loading Basis Set <=

    Name: 6-311G(D,P)
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   151 file /usr/local/share/psi4/basis/6-311g_d_p_.gbs 
    atoms 2-3 entry H          line    32 file /usr/local/share/psi4/basis/6-311g_d_p_.gbs 

   => M06-2X-D3: Empirical Dispersion <=

    Grimme's -D3 (zero-damping) Dispersion Correction
    Grimme S.; Antony J.; Ehrlich S.; Krieg H. (2010), J. Chem. Phys., 132: 154104
    Parametrisation from:    L. Goerigk, S. Grimme, Phys. Chem. Chem. Phys. 13, 6670-6688, 2011

        s6 =       1.000000
        s8 =       0.000000
       sr6 =       1.619000
    alpha6 =      14.000000
       sr8 =       1.000000


         ----------------------------------------

In [16]:
xyz = mol.save_string_xyz()
print (xyz)

0 1
 O    0.000000000003   -0.000000000000    0.065991238612
 H    0.756279797719   -0.000000000000   -0.523664422665
 H   -0.756279797768    0.000000000000   -0.523664422608



In [17]:
psi4.core.set_output_file("frq.out")

psi4.set_memory('10GB')
psi4.set_num_threads(2)

# save molden file for Avogadro (GUI)
psi4.set_options({"NORMAL_MODES_WRITE": True})

# calculation (vibration)
#scf_e, wfn = psi4.frequency("B3LYP-D3BJ/6-311G(d,p)", ref_gradient = wfn.gradient(), return_wfn = True)
#scf_e, wfn = psi4.frequency("M06-D3/6-311G(d,p)", ref_gradient = wfn.gradient(), return_wfn = True) # metal
scf_e, wfn = psi4.frequency("M06-2X-D3/6-311G(d,p)", ref_gradient = wfn.gradient(), return_wfn = True) # non-metal

 7 displacements needed.
 1 2 3 4 5 6 7


In [18]:
!cat frq.out


  Memory set to   9.313 GiB by Python driver.
  Threads set to 2 by Python driver.
hessian() using ref_gradient to assess stationary point.

  Based on options and gradient (rms=1.55E-05), recommend projecting translations and projecting rotations.
hessian() will perform frequency computation by finite difference of analytic gradients.

         ----------------------------------------------------------
                                   FINDIF
                     R. A. King and Jonathon Misiewicz
         ---------------------------------------------------------

  Using finite-differences of gradients to determine vibrational frequencies and 
  normal modes. Resulting frequencies are only valid at stationary points.
    Generating geometries for use with 3-point formula.
    Displacement size will be 5.00e-03.
    Number of atoms is 3.
    Number of irreps is 1.
    Number of SALCs is 3.
    Translations projected? 1. Rotations projected? 1.
    Index of SALCs per irrep:
     1 :  

In [19]:
wfn.frequencies().to_array()
wfn.hessian().to_array()

array([[ 6.98103792e-01,  0.00000000e+00, -8.44423279e-10,
        -3.49051896e-01,  0.00000000e+00,  2.72148519e-01,
        -3.49051896e-01,  0.00000000e+00, -2.72148518e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-8.44423279e-10,  0.00000000e+00,  4.73244563e-01,
         2.13407817e-01,  0.00000000e+00, -2.36622282e-01,
        -2.13407816e-01,  0.00000000e+00, -2.36622281e-01],
       [-3.49051896e-01,  0.00000000e+00,  2.13407817e-01,
         3.79906648e-01,  0.00000000e+00, -2.42778168e-01,
        -3.08547512e-02,  0.00000000e+00,  2.93703510e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.72148519e-01,  0.00000000e+00, -2.36622282e-01,
        -2.42778168e-01,  0.00000000e+00,  2.244054

In [20]:
import numpy as np
print(f'Frequencies:\n{np.array(wfn.frequencies())}')
print(f'Frequencies:\n{np.array(wfn.hessian())}')

Frequencies:
[1626.72626319 3887.11064437 3983.55642662]
Frequencies:
[[ 6.98103792e-01  0.00000000e+00 -8.44423279e-10 -3.49051896e-01
   0.00000000e+00  2.72148519e-01 -3.49051896e-01  0.00000000e+00
  -2.72148518e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-8.44423279e-10  0.00000000e+00  4.73244563e-01  2.13407817e-01
   0.00000000e+00 -2.36622282e-01 -2.13407816e-01  0.00000000e+00
  -2.36622281e-01]
 [-3.49051896e-01  0.00000000e+00  2.13407817e-01  3.79906648e-01
   0.00000000e+00 -2.42778168e-01 -3.08547512e-02  0.00000000e+00
   2.93703510e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.72148519e-01  0.00000000e+00 -2.36622282e-01 -2.42778168e-01
   0.00000000e+00  2.24405417e-01 -2.93703511e-02  0.00000000e+00
   1.22168651e-02]
 [-3.49051896e-01  0.000